In [ ]:
pip install python-dotenv

In [ ]:
pip install openai

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

#print('key:', OPENAI_API_KEY)

In [ ]:
from openai import OpenAI

client =  OpenAI(api_key=OPENAI_API_KEY)

response = client.responses.create(
    model='gpt-4.1',
    input='스타워즈 시리즈의 다스베이더의 역할에 대해 한줄로 설벙해줘'
)

In [5]:
response.output_text

'다스 베이더는 스타워즈 시리즈에서 제다이에서 시스가 된 강력한 다크사이드 전사이자, 은하 제국의 핵심 인물이다.'

In [6]:
response = client.responses.create(
    model='gpt-4.1',
    instructions='당신은 영화평론가야!',
    input='스타워즈 시리즈의 다스베이더의 역할에 대해 한줄로 설벙해줘'
)
print(response.output_text)

다스 베이더는 스타워즈 시리즈에서 타락한 영웅이자, 은하 제국의 잔혹한 집행자로서 아버지와 적, 그리고 구원의 상징이라는 복합적인 역할을 수행한다.


In [7]:
response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role': 'developer',
            'content': '음식에 대한 이야기 하는것을 좋아해'
        },
        {
            'role': 'user',
            'content': '오늘은 무엇을 먹을까?'
        }
    ]
)
print(response.output_text)

오늘 어떤 기분이신가요? 든든하게 한식 한 끼? 아니면 가볍게 샐러드나 샌드위치? 혹시 특별히 땡기는 음식이 있으신가요?  
예를 들어,

- 든든한 한식: 김치찌개, 제육볶음, 불고기, 된장찌개 등등  
- 다양한 양식: 파스타, 피자, 스테이크  
- 건강하게: 샐러드, 닭가슴살, 연어  
- 간편하게: 김밥, 샌드위치, 라면, 만두  

아니면, 오늘 날씨나 기분에 따라 추천해 드릴 수도 있어요! 알려주시면 맞춤 추천 드릴게요 😊  
궁금한 음식이나 처음 도전해보고 싶은 음식 있으면 알려주세요!


In [8]:
completion = client.chat.completions.create(
    model='gpt-4.1',
    messages= [
        {
            'role': 'user',
            'content': '스타워즈시리즈에 대해 한분장으로 설명해줘'
        }
    ]  
)


In [13]:
#dict(completion.choices[0])
completion.choices[0].message.content

'스타워즈 시리즈는 먼 우주의 은하계를 배경으로 제다이와 시스 등 다양한 세력 간의 갈등, 영웅과 악당들의 운명, 권력과 희생, 가족과 우정의 서사를 그린 SF 판타지 영화 프랜차이즈입니다.'

In [16]:
user_input = input('안녕하세요. 무엇이 굼금하신가요? 질문을 남겨주세요!!!')
print('질문:', user_input)

response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            'role': 'developer',
            'content': '너는 유치원 선생님이야. 아이들을 대하는것처럼 이야기해줘'
        },
        {
            'role': 'user',
            'content': user_input
        }
    ]
)
print(response.output_text)

질문: 미니언즈는 어디에 사나요?
우리 귀여운 미니언즈 친구들은 영화 속에서 주로 "그루"라는 악당과 함께 살고 있어요! 미니언즈는 특별한 나라에 살지 않아요. 영화 속에서는 사막, 남극, 그리고 여러 신기한 곳들을 여행하지만, 실제로는 우리 마음 속에서 살아가고 있답니다.

미니언즈는 바나나를 아주 좋아하고 항상 장난을 좋아해서, 어디서든 재미있게 놀면서 살아가는 친구들이에요. 우리도 미니언즈처럼 친구들과 사이좋게 지내고, 재미있게 놀아요!


In [17]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [19]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
    "required": ["latitude", "longitude"],
    "additionalProperties": False
    },
    "strict": True
}]

In [ ]:
input_messages = [{ 'role': 'user', 'content':'오늘 파리 날씨 어때?'}]

# 함수 호출하는 기능
response = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

In [24]:
dict(response.output[0])

{'arguments': '{"latitude":48.8566,"longitude":2.3522}',
 'call_id': 'call_5akC5g1dyEDCKEpL40sMoiBY',
 'name': 'get_weather',
 'type': 'function_call',
 'id': 'fc_685bae49a6608198832c554beffde293067ceb33fda1e78d',
 'status': 'completed'}

In [25]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args["latitude"], args["longitude"])
print(result)

23.3


In [26]:
input_messages.append(tool_call)
input_messages.append({
    'type' : 'function_call_output',
    'call_id': tool_call.call_id,
    'output': str(result)
})

# 최종 응답
response2 = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

print(response2.output_text)

오늘 파리의 현재 기온은 약 23.3°C입니다. 날씨가 꽤 온화하고 쾌적한 편이네요! 더 자세한 정보(예: 강수나 바람 등)가 필요하시면 말씀해 주세요.
